In [74]:
import numpy as np
from collections import defaultdict 
from statistics import mean, stdev
import pandas as pd
from math import pi, sqrt, exp, log
import operator

## Preprocess Function

In [58]:
## Preprocess function

# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing
# The instances where all the attributes are missing are removed, and mean the mean of the attributes in each class is imputed.

def preprocess(dataname):
    
    attribute_list = ["pose","x1","x2","x3","x4","x5","x6","x7","x8","x9","x10","x11","y1","y2","y3","y4","y5","y6","y7","y8","y9","y10","y11"]
    df = pd.read_csv(dataname, names = attribute_list)
    
    # replace all 9999 by nan
    df.replace(9999, np.nan, inplace=True)

    # get rid of fully identifiable instance (row of nan)
    df.dropna(subset=df.columns.difference(['pose']), how='all', inplace = True)
    
    
    # Q5 mean imputation code
    poses = df.pose.unique()
    for attribute in attribute_list[1:]:
        df[attribute] = df[attribute].fillna(df.groupby('pose')[attribute].transform('mean'))
    
    return df

## Train Function

In [59]:
## Train function

# This function calculates and returns a dictionary of the mean, standard deviation and size of the instances
# which is later used when calculating the posteriors and likelihoods

def train(traindata):

    df_groups = traindata.groupby(traindata.pose)
    
    # get lists of the mean, standard deviation and group size of each pose (class)
    mean_dict = df_groups.mean().T.to_dict()
    std_dict = df_groups.std().T.to_dict()
    group_size = df_groups.size().to_dict()
    
    summary_dict = defaultdict(list)
    
    for pose in mean_dict:
        for loc in mean_dict[pose]:
            summary_dict[pose].append([mean_dict[pose][loc],std_dict[pose][loc], group_size[pose]])
    
    return summary_dict
    

In [60]:
# Calculates and returns a dictionary of the posterior probability of each class.

def posterior_per_class(summary, instance, row_total):
    
    posteriors = dict()
    for pose in summary:
        
        # get the ratio of the class to the whole number of instances
        posterior = log(summary[pose][0][2]/row_total)
        for i in range(len(instance)-1):
            
            # add the logs of the posteriors to the dictionary of posteriors
            if(gaussian_probability(instance[i], summary[pose][i][0], summary[pose][i][1]) > 0):
                posterior += log(gaussian_probability(instance[i], summary[pose][i][0], summary[pose][i][1]))
        posteriors[pose] = posterior
        posterior = 0
        
    return posteriors

In [61]:
# Function to calculate the gaussian probability

def gaussian_probability(x, mean, std):
    
    return 1/(std*sqrt(2*pi)) * exp(-((x-mean)**2/(2*std**2)))

## Predict function

In [62]:
# Function that makes a list of the predictions for the test data instances based on the parameters obtained from the train data

def predict(traindata, testdata):
    
    # get the total number of instances in the test data
    row_total = testdata.shape[0]
    
    # convert the dataframe into a list of instances
    instance_list = testdata.values.tolist()
    
    predictions = list()
    
    # Get the summary of the train data to use for the posteriors
    trained_summary = train(traindata)
    
    for instance in instance_list:
        posteriors = posterior_per_class(trained_summary, instance[1:], row_total)
        
        # Get the class with the highest posterior for the instance to use as a prediction
        predictions.append(max(posteriors.items(), key=operator.itemgetter(1))[0])
        
    return predictions

In [63]:
# Function that returns a dictionary of the precision and recalls of each pose (class)

def precision_and_recall(actual_poses, prediction):
    
    poses = list()
    
    # get list of poses
    for pose in actual_poses:
        if pose not in poses:
            poses.append(pose)
    
    eval_dict = defaultdict(list)
    
    TP = 0
    FP = 0
    FN = 0
    
    for pose in poses:
        for i in range(len(actual_poses)):
            
            # get the TP, FP and FN by comparing each pose in actual poses and the predicted poses
            if(pose == actual_poses[i] and pose == prediction[i]):
                TP += 1
            elif(pose != actual_poses[i] and pose == prediction[i]):
                FP += 1
            elif(pose == actual_poses[i] and pose != prediction[i]):
                FN += 1
        eval_dict[pose].append(TP/(TP + FP))
        eval_dict[pose].append(TP/(TP + FN))
        TP = 0
        FP = 0
        FN = 0
    
    return eval_dict

In [64]:
# Function that computes the correct predictions out of the total number of predictions

def accuracy(actual_poses, prediction):
    
    correct = 0
    for i in range(len(actual_poses)):
        if (actual_poses[i] == prediction[i]):
            correct += 1
    
    return correct/len(actual_poses)

**Related to Q1**

In [65]:
# Function that computes the macro average of the perforance of the classifier

def macro_averaging(eval_dict):
    
    p_r_sum = [0, 0]
    
    # Number of classes
    no_class = 0
    
    for pose in eval_dict:
        no_class += 1
        
        # Precision sum
        p_r_sum[0] += eval_dict[pose][0]
        # Recall sum
        p_r_sum[1] += eval_dict[pose][1]
    
    p_r_mean = [x/no_class for x in p_r_sum]
    
    return p_r_mean

**Related to Q1**

In [66]:
## related to Q1
# Function that computes the macro average of the perforance of the classifier
# implements logic from lecture slide 37 of Model Evaluation

def micro_averaging(actual_poses, prediction):
    
    poses = list()
    
    # get list of poses
    for pose in actual_poses:
        if pose not in poses:
            poses.append(pose)
    
    TP = 0
    FP = 0
    FN = 0
    
    p_r_dict = defaultdict(list)
    
    for pose in poses:
        for i in range(len(actual_poses)):
            if(pose == actual_poses[i] and pose == prediction[i]):
                TP += 1
            elif(pose != actual_poses[i] and pose == prediction[i]):
                FP += 1
            elif(pose == actual_poses[i] and pose != prediction[i]):
                FN += 1
        p_r_dict[pose] = [TP, FP, FN]
        TP = 0
        FP = 0
        FN = 0
    
    TPsum = 0
    TPFP = 0
    TPFN = 0
    
    for pose in p_r_dict:
        TPsum += p_r_dict[pose][0]
        TPFP += p_r_dict[pose][0] + p_r_dict[pose][1]
        TPFN += p_r_dict[pose][0] + p_r_dict[pose][2]
        
    return [TPsum/TPFP, TPsum/TPFN]

**Related to Q1**

In [73]:
# Function that computes the F1-score of the each evaluation metric calculation

def f1score(precision_recall):
    
    P = precision_recall[0]
    R = precision_recall[1]
    return ((2*P*R)/(P + R))

## Evaluate Function

In [71]:
# Function that evaluates the performance of the classifier, comparing the actual data and predictions
# 'A scoring function'

def evaluate(testdata, prediction):
    
    actual_poses = testdata['pose'].tolist()

    print("accuracy: ", accuracy(actual_poses, prediction))
    
    eval_dict = precision_and_recall(actual_poses, prediction)
    
    macro = macro_averaging(eval_dict)
    micro = micro_averaging(actual_poses, prediction)
    print("Macro-average Precision:", macro[0], "Macro-average Recall:", macro[1], "F1-score:", f1score(macro))
    print("Micro-average Precision:", micro[0], "Micro-average Recall:", micro[1], "F1-score:", f1score(micro))
    
    return

In [69]:
preprocessed_train = preprocess('train.csv')
preprocessed_test = preprocess('test.csv')

print('train on train (train data used as test data):')
evaluate(preprocessed_train, predict(preprocessed_train, preprocessed_train))
print('\n')
print('test:')
evaluate(preprocessed_test, predict(preprocessed_train, preprocessed_test))




train on train (train data used as test data):
accuracy:  0.8989071038251366
Macro-average Precision: 0.9011067359560316 Macro-average Recall: 0.9016762383725847 F1-score: 0.9013913972108842
Micro-average Precision: 0.8989071038251366 Micro-average Recall: 0.8989071038251366 F1-score: 0.8989071038251366


test:
accuracy:  0.7678571428571429
Macro-average Precision: 0.7567136784783843 Macro-average Recall: 0.7523412698412699 F1-score: 0.7545211397459243
Micro-average Precision: 0.7678571428571429 Micro-average Recall: 0.7678571428571429 F1-score: 0.7678571428571429


### Q1
Since this is a multiclass classification problem, there are multiple ways to compute precision, recall, and F-score for this classifier. Implement at least two of the methods from the "Model Evaluation" lecture and discuss any differences between them. (The implementation should be your own and should not just call a pre-existing function.)

In [ ]:
# The macro_averaging(), micro_averaging() and f1score() function are all related to this question.

### Q5
Naive Bayes ignores missing values, but in pose recognition tasks the missing values can be informative. Missing values indicate that some part of the body was obscured and sometimes this is relevant to the pose (e.g., holding one hand behind the back). Are missing values useful for this task? Implement a method that incorporates information about missing values and demonstrate whether it changes the classification results.

In [ ]:
# A mean imputation was additionally performed in the preprocessing function as a way of incorporating information about the missing values.